## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-20-21-47-05 +0000,nyt,Judge Temporarily Blocks Trump From Deploying ...,https://www.nytimes.com/2025/11/20/us/politics...
1,2025-11-20-21-45-00 +0000,nyt,Trump to Send U.S. Official to G20 in South Af...,https://www.nytimes.com/2025/11/20/world/afric...
2,2025-11-20-21-44-35 +0000,nyt,Stocks Tumble as Early Tech Rally Gives Way to...,https://www.nytimes.com/2025/11/19/business/st...
3,2025-11-20-21-43-00 +0000,wsj,Judge Rules National Guard Deployment in Washi...,https://www.wsj.com/us-news/law/judge-rules-na...
4,2025-11-20-21-43-00 +0000,wsj,Bath & Body Works CEO Says Former Strategy Fai...,https://www.wsj.com/business/retail/bath-body-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2329/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
3,trump,48
567,new,17
137,house,17
672,epstein,14
145,ukraine,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
324,2025-11-20-02-34-06 +0000,nypost,Trump reveals when he will meet NYC Mayor-elec...,https://nypost.com/2025/11/19/us-news/trump-re...,131
338,2025-11-20-01-41-44 +0000,wapo,U.S. pushes new Ukraine peace plan as Trump’s ...,https://www.washingtonpost.com/national-securi...,114
325,2025-11-20-02-32-00 +0000,wsj,Trump Says He Will Meet With Mamdani Friday at...,https://www.wsj.com/politics/elections/trump-s...,108
267,2025-11-20-09-59-52 +0000,cbc,Trump to meet NYC mayor-elect Mamdani at White...,https://www.cbc.ca/news/world/trump-to-meet-ny...,108
320,2025-11-20-02-48-43 +0000,nyt,Trump and Mamdani Will Meet at White House on ...,https://www.nytimes.com/2025/11/19/nyregion/tr...,103


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
324,131,2025-11-20-02-34-06 +0000,nypost,Trump reveals when he will meet NYC Mayor-elec...,https://nypost.com/2025/11/19/us-news/trump-re...
312,87,2025-11-20-03-35-00 +0000,wsj,Congress’s approval of legislation to force a ...,https://www.wsj.com/politics/policy/epstein-fi...
136,63,2025-11-20-17-20-17 +0000,nypost,Rubio casts doubt on pro-Russian Ukraine peac...,https://nypost.com/2025/11/20/world-news/rubio...
135,63,2025-11-20-17-20-47 +0000,nypost,Kamala Harris and Jill Biden come face-to-face...,https://nypost.com/2025/11/20/us-news/kamala-h...
190,43,2025-11-20-14-18-22 +0000,nyt,Israeli Strikes Kill at Least 25 in Gaza Amid ...,https://www.nytimes.com/2025/11/19/world/middl...
191,41,2025-11-20-14-07-21 +0000,nypost,Delayed government data show US added surprisi...,https://nypost.com/2025/11/20/business/delayed...
282,41,2025-11-20-08-35-04 +0000,wsj,Stock Market Today: Stock-Market Rebound Evapo...,https://www.wsj.com/livecoverage/jobs-report-b...
111,38,2025-11-20-18-05-00 +0000,wsj,Former President George W. Bush hailed Dick Ch...,https://www.wsj.com/politics/vice-president-di...
295,37,2025-11-20-06-36-15 +0000,nypost,Terminally ill 25-year-old chooses to die on ‘...,https://nypost.com/2025/11/20/world-news/termi...
114,35,2025-11-20-17-58-03 +0000,nypost,New photos of UPS MD-11 jet crash show engine ...,https://nypost.com/2025/11/20/us-news/new-phot...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
